<a href="https://colab.research.google.com/github/shrikantvarma/AgenticAI/blob/main/Basic_Agent_flow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ReAct Agent with Tool Use in Google Colab

This repository contains a Google Colab notebook demonstrating a simple implementation of a ReAct (Reasoning and Acting) agent that can use external tools via the OpenAI API.

## What is a ReAct Agent?

ReAct is a pattern that combines **Reasoning** (generating thoughts to determine the next step) and **Acting** (interacting with external environments through tools). The agent interleaves these steps in a loop to solve tasks.

## Features

*   Implements the core ReAct loop (Thought, Action, Observation).
*   Uses the OpenAI API for the language model's reasoning.
*   Includes examples of tool definitions for:
    *   Fetching fake weather data.
    *   Evaluating mathematical expressions.
    *   Searching for recent news on the web using DuckDuckGo.
*   Designed to run in a Google Colab environment.

## Setup

1.  **Open the Notebook:** Open the provided Google Colab notebook in your browser.
2.  **Get an OpenAI API Key:** Obtain an API key from the [OpenAI website](https://platform.openai.com/).
3.  **Add API Key to Colab Secrets:** In the left-hand sidebar of Google Colab, click the "🔑 Secrets" tab. Add a new secret with the key name `OPENAI_API_KEY` and paste your OpenAI API key as the value.
4.  **Install Dependencies:** Run the cell containing the `!pip install` command to install the necessary Python libraries (`openai` and `duckduckgo-search`).
5.  **Run Cells:** Execute the notebook cells sequentially to define the tools, system prompt, and the `react_agent` function.

## How it Works

The `react_agent` function is the core of the implementation. It:

1.  Initializes the conversation with a system prompt explaining the ReAct pattern.
2.  Enters a loop for a predefined number of steps.
3.  Sends the current conversation history to the OpenAI model.
4.  Parses the model's response for a "Thought:", "Action:", or "Final Answer:".
5.  If an "Action:" is detected, it identifies the tool name and arguments.
6.  It then calls the corresponding Python function defined as a tool.
7.  The result of the tool call ("Observation") is added back to the conversation history.
8.  If a "Final Answer:" is detected, the process concludes.

The implementation includes logic to parse tool calls from the model's text output, mimicking the original ReAct paper's style, as well as handling structured tool calls if the model provides them.

## Tools

The following tools are defined and available for the agent to use:

*   **`get_weather(city: str)`**: Returns fake weather data for a few predefined cities.
*   **`calculate_expression(expression: str)`**: Safely evaluates a given mathematical expression.
*   **`search_web_news(query: str, max_results: int = 3)`**: Searches for recent news articles based on a query using the DuckDuckGo search engine and returns the top results.

Each tool has a corresponding schema defined in the `tools` list, which is provided to the OpenAI API so the model knows about the available tools and their parameters.

## Usage

Once the setup is complete and all necessary cells have been run, you can use the `react_agent` function to interact with the agent:

In [2]:
import warnings

# Ignore DeprecationWarning specifically from the jupyter_client module
# warnings.filterwarnings("ignore", category=DeprecationWarning, module="jupyter_client")

# You can also ignore all DeprecationWarnings (use with caution)
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [ ]:
!pip install openai duckduckgo-search

In [7]:
from openai import OpenAI
from duckduckgo_search import DDGS
import json

In [8]:
from google.colab import userdata
openai_api_key = userdata.get('OPENAI_API_KEY').strip()
client = OpenAI(api_key=openai_api_key)

In [9]:
# -------------------------------------------------------
# Define the tools (functions the agent can call)
# -------------------------------------------------------

def get_weather(city: str):
    """Return fake weather data for demo."""
    data = {
        "Paris": {"temperature": "23°C", "condition": "Sunny"},
        "London": {"temperature": "18°C", "condition": "Cloudy"},
        "Tokyo": {"temperature": "27°C", "condition": "Humid"},
    }
    return data.get(city, {"temperature": "Unknown", "condition": "Unknown"})


def calculate_expression(expression: str):
    """Safely evaluate a math expression."""
    try:
        result = eval(expression, {"__builtins__": {}})
        return {"result": result}
    except Exception as e:
        return {"error": str(e)}

def search_web_news(query: str, max_results: int = 3):
    """Use DuckDuckGo to search the web and return top news results."""
    results = []
    with DDGS() as ddgs:
        # Use the news() method for potentially more relevant news results
        for r in ddgs.news(query, max_results=max_results):
            results.append({"title": r["title"], "snippet": r["body"], "url": r["url"]}) # Note: news() returns 'url' instead of 'href'
    return results

# Map tool names → Python functions
tool_functions = {
    "get_weather": get_weather,
    "calculate_expression": calculate_expression,
    "search_web_news": search_web_news,
}

In [10]:
# -------------------------------------------------------
# Define the tool schemas (so LLM knows they exist)
# -------------------------------------------------------
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Get the current weather for a city.",
            "parameters": {
                "type": "object",
                "properties": {
                    "city": {"type": "string", "description": "Name of the city"},
                },
                "required": ["city"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "calculate_expression",
            "description": "Perform a basic math calculation.",
            "parameters": {
                "type": "object",
                "properties": {
                    "expression": {
                        "type": "string",
                        "description": "Math expression like '12*(4+7)'",
                    },
                },
                "required": ["expression"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "search_web_news",
            "description": "Search the web for recent news.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {"type": "string", "description": "Search query"},
                    "max_results": {
                        "type": "integer",
                        "description": "Number of search results to retrieve",
                    },
                },
                "required": ["query"],
            },
        },
    },
]

In [11]:
# -------------------------------------------------------
# Explicit ReAct System Prompt
# -------------------------------------------------------
SYSTEM_PROMPT = """You are an intelligent ReAct-style reasoning assistant.

Follow this loop until you can give the final answer:

Thought: reason about what to do next.
Action: if a tool is needed, call one using the exact JSON arguments.
Observation: you will receive the tool’s result.
Final Answer: only once you have enough information.

Use this format:
Thought: ...
Action: ...
Observation: ...
Thought: ...
Final Answer: ...

Always think step by step, and only call tools when necessary.
"""

In [12]:
# -------------------------------------------------------
# Core ReAct loop
# -------------------------------------------------------
import json
import re # Import the regex module

def react_agent(user_query, model="gpt-4o-mini", max_steps=5):
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": user_query},
    ]

    print(f"\n🧠 User asked: {user_query}")

    for step in range(max_steps):
        # Debugging: Print messages being sent
        # print(f"\n--- Messages being sent at Step {step+1} ---")
        # for message in messages:
        #     print(message)
        # print("----------------------------")

        response = client.chat.completions.create(
            model=model,
            messages=messages,
            tools=tools
        )
        # Debugging: Print entire response object
        # print(f"\n--- Response received at Step {step+1} ---")
        # print(response)
        # print("----------------------------")


        message = response.choices[0].message
        content = message.content or ""

        # 🧩 Print model reasoning (Thought/Action/Observation) - Keep this as it's part of ReAct output
        if content:
            print(f"\n🧩 Model Output Step {step+1}:\n{content.strip()}")

        tool_call = None
        func_name = None
        args = None
        simulated_tool_call_id = None # Initialize simulated ID

        # Check for structured tool calls first (preferred)
        if hasattr(message, "tool_calls") and message.tool_calls:
            tool_call = message.tool_calls[0]
            func_name = tool_call.function.name
            args = json.loads(tool_call.function.arguments)
            # Debugging: Indicate structured tool call detection
            # print(f"\n🤖 Model provided structured tool call: `{func_name}` with args: {args}")
            # For structured calls, the tool_call object already has the ID
            simulated_tool_call_id = tool_call.id


        # If no structured tool calls, try parsing from content (ReAct style)
        elif "Action:" in content:
            # Debugging: Indicate attempt to parse from content
            # print("\n🔍 Attempting to parse Action from content...")
            # Updated regex to handle "Action: {"key": "value"}" or "Action: tool_name({"key": "value"})"
            match_json_args = re.search(r"Action:\s*(?:(\w+)\s*)?(\{.*\})", content)
            match_func_args = re.search(r"Action:\s*(\w+)\s*\((.*?)\)", content) # Keep old regex for func(arg=val) format

            if match_json_args:
                func_name = match_json_args.group(1) # Optional function name before JSON
                args_str = match_json_args.group(2)
                try:
                    args = json.loads(args_str)
                    # If function name wasn't explicitly provided before JSON, try to infer from args if possible
                    if func_name is None and 'city' in args:
                         func_name = 'get_weather'
                         # Debugging: Indicate inferred function name
                         # print("✅ Parsed Action: Inferred `get_weather` from args.")
                    elif func_name is None and 'query' in args:
                         func_name = 'search_web' # Assuming search_web or search_web_news
                         # Need to be careful here, search_web is old name, search_web_news is new.
                         # Use search_web_news as that's the current tool name.
                         func_name = 'search_web_news'
                         # Debugging: Indicate inferred function name
                         # print("✅ Parsed Action: Inferred `search_web_news` from args.")


                    if func_name:
                         # Debugging: Indicate parsed action with args
                         # print(f"✅ Parsed Action: `{func_name}` with JSON args: {args}")
                         simulated_tool_call_id = f"parsed_call_{step}" # Create a unique ID for the parsed call

                    else:
                         # Debugging: Indicate failure to determine function name
                         # print("❌ Could not determine function name from JSON action.")
                         func_name = None # Invalidate if function name can't be determined


                except json.JSONDecodeError as e:
                     # Debugging: Indicate JSON parsing failure
                     # print(f"❌ Failed to parse JSON arguments '{args_str}': {e}")
                     func_name = None # Invalidate the parsed action if args parsing fails
                except Exception as e:
                     # Debugging: Indicate unexpected parsing error
                     # print(f"❌ An unexpected error occurred during JSON parsing: {e}")
                     func_name = None

            elif match_func_args: # Fallback to old regex if JSON format not matched
                 func_name = match_func_args.group(1)
                 args_str = match_func_args.group(2)
                 try:
                     # Attempt to parse old style key=value args
                     args_dict = {}
                     if args_str:
                          arg_matches = re.findall(r'(\w+)=["\']?(.*?)["\']?(?:,\s*)?', args_str)
                          for key, value in arg_matches:
                              args_dict[key] = value
                          if not args_dict and args_str.strip():
                              # Debugging: Indicate treating as single query arg
                              # print("Treating as single query argument.")
                              args_dict['query'] = args_str.strip()

                     args = args_dict
                     # Debugging: Indicate parsed action (old format)
                     # print(f"✅ Parsed Action (old format): `{func_name}` with args: {args}")
                     simulated_tool_call_id = f"parsed_call_{step}" # Create a unique ID for the parsed call


                 except Exception as e:
                      # Debugging: Indicate old format parsing failure
                      # print(f"❌ Failed to parse arguments (old format) '{args_str}': {e}")
                      func_name = None

            else:
                 # Debugging: Indicate pattern not matched
                 # print("❌ Could not parse Action: pattern not matched.")
                 pass # Do nothing if pattern not matched, allows loop to check for Final Answer


        # If a tool call (structured or parsed) was identified AND a simulated ID was created
        if func_name and simulated_tool_call_id:
             if func_name in tool_functions:
                 # Debugging: Indicate tool invocation
                 # print(f"🤖 Invoking tool `{func_name}` with args: {args}")
                 result = tool_functions[func_name](**args)
                 # Debugging: Indicate observation
                 # print(f"🧾 Observation from {func_name}: {result}")

                 # Feed observation back
                 # When parsing from content, construct a message that mimics a structured tool call
                 messages.append({
                     "role": "assistant",
                     "content": content, # Include the original content
                     "tool_calls": [{ # Add the simulated tool_calls structure
                         "id": simulated_tool_call_id,
                         "function": {
                             "name": func_name,
                             "arguments": json.dumps(args) # Arguments need to be a JSON string here
                         },
                         "type": "function"
                     }]
                 })
                 messages.append({"role": "tool", "tool_call_id": simulated_tool_call_id, "content": str(result)})
                 continue # go to next step

             else:
                 # Debugging: Indicate unknown tool
                 # print(f"⚠️ Unknown tool: {func_name}")
                 break


        # If model gives final answer AND no tool calls (structured or parsed) were made in this step
        # The check for func_name implies no tool call was successfully identified/parsed
        if "Final Answer:" in content and not func_name:
             print(f"\n✅ Final Answer:\n{content}")
             break

        # If no tool calls (structured or parsed) and no final answer yet
        if not func_name:
             # Debugging: Indicate no tool call or final answer
             # print("\n⚠️ Model did not provide a tool call or a final answer in this step.")
             break

In [13]:
react_agent("What's the weather like in Paris?")



🧠 User asked: What's the weather like in Paris?

🧩 Model Output Step 1:
Thought: I need to check the current weather in Paris to provide an accurate answer. 
Action: {"city":"Paris"} 
Observation: ...

🧩 Model Output Step 2:
Thought: I have the current weather information for Paris, which is 23°C and sunny. 
Final Answer: The weather in Paris is currently 23°C and sunny.

✅ Final Answer:
Thought: I have the current weather information for Paris, which is 23°C and sunny. 
Final Answer: The weather in Paris is currently 23°C and sunny.


In [14]:
react_agent("Give me the latest AI news")


🧠 User asked: Give me the latest AI news


/tmp/ipython-input-3113594215.py:26: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use time


🧩 Model Output Step 2:
Observation: I have retrieved the latest AI news articles. Here are the summaries:

1. **The World's First AI-Powered Minister Tests the Future of Government**  
   An AI system named Diella has joined Albania's cabinet, testing how far governments are willing to trust machines with power. [Read more](https://www.msn.com/en-us/news/world/the-first-ai-powered-minister-tests-the-future-of-government/ar-AA1OefCU)

2. **'I Believe It's a Bubble': What Some Smart People Are Saying About AI**  
   A growing group of critics say we're in an artificial intelligence bubble. Is it true? If so, how would we know? [Read more](https://www.msn.com/en-us/money/technology/i-believe-it-s-a-bubble-what-some-smart-people-are-saying-about-ai/ar-AA1ObeBI)

3. **MGB is turning to AI to ease shortage of primary care doctors. Some of them don't like it.**  
   Several providers said the app — which questions patients, reviews medical records, and produces a list of potential diagnoses 

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

In [ ]:
react_agent("Calculate 15 * (10 + 5)")

In [ ]:
react_agent("Give me the latest news on artificial intelligence.")